<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/stock_screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
import datetime

In [2]:
today = datetime.date.today().strftime('%Y%m%d')

In [3]:
data = {
    'mktId': 'KSQ',
    'trdDd': today,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901',
}
gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_key = requests.post(gen_url, data=data)

In [4]:
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
r = requests.post(down_url, data={'code':gen_key.text})
r.encoding = 'EUC-KR'

In [5]:
from io import StringIO

df = pd.read_csv(StringIO(r.text))

In [6]:
df

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,060310,3S,KOSDAQ,기계·장비,3985,80,2.05,184391979305
1,054620,APS홀딩스,KOSDAQ,금융,17600,50,0.28,358938289600
2,265520,AP시스템,KOSDAQ,반도체,26900,-800,-2.89,389518106300
3,211270,AP위성,KOSDAQ,통신장비,17900,200,1.13,269973241600
4,013720,CBI,KOSDAQ,운송장비·부품,3415,135,4.12,173132901625
...,...,...,...,...,...,...,...,...
1508,024060,흥구석유,KOSDAQ,유통,8020,600,8.09,120300000000
1509,010240,흥국,KOSDAQ,기계·장비,9670,540,5.91,119160470320
1510,189980,흥국에프엔비,KOSDAQ,음식료·담배,3280,95,2.98,125909776560
1511,037440,희림,KOSDAQ,기타서비스,7110,460,6.92,98988797250


In [7]:
%%timeit
df.loc[df['시가총액'].between(30000000000,70000000000), '종목명']

The slowest run took 13.31 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 5: 573 µs per loop


In [8]:
%%timeit
df[df['시가총액'].between(30000000000,70000000000)]['종목명']

1000 loops, best of 5: 925 µs per loop


In [9]:
boundary = df.loc[df['시가총액'].between(30000000000,70000000000)]
boundary

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
10,056000,COWON,KOSDAQ,디지털컨텐츠,2860,0,0.00,56399843500
12,083660,CSA 코스믹,KOSDAQ,화학,1515,5,0.33,57470182710
20,050120,ES큐브,KOSDAQ,유통,4655,-85,-1.79,63140820330
22,900290,GRT,KOSDAQ,금융,951,0,0.00,64073625000
35,099520,ITX-AI,KOSDAQ,통신장비,1285,0,0.00,37652662655
...,...,...,...,...,...,...,...,...
1461,079170,한창산업,KOSDAQ,금속,9710,290,3.08,50492000000
1464,002680,한탑,KOSDAQ,음식료·담배,1415,5,0.35,40002033020
1473,900270,헝셩그룹,KOSDAQ,금융,506,4,0.80,40480000000
1486,011080,형지I&C,KOSDAQ,섬유·의류,1465,95,6.93,57070053620


In [10]:
!pip install OpenDartReader

In [11]:
import OpenDartReader

api_key = '4b0512c473e52e1cf59f8ce8380b2b30904f381a'
dart = OpenDartReader(api_key) 

In [12]:
search_date = '2021-08-20'

In [13]:
dart_df = dart.list(start=search_date, kind='A')
found = list(set(dart_df['stock_code']).intersection(set(boundary['종목코드'])))
out_df = dart_df[dart_df['stock_code'].isin(found)]

In [14]:
out_df

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
4,00818472,이큐셀,160600,K,사업보고서 (2021.06),20210917000607,이큐셀,20210917,연
5,00814786,스카이이앤엠,131100,K,사업보고서 (2021.06),20210917000602,스카이이앤엠,20210917,
9,00939942,포시에스,189690,K,[기재정정]사업보고서 (2021.06),20210917000423,포시에스,20210917,연
12,00138747,아세아텍,050860,K,사업보고서 (2021.06),20210917000175,아세아텍,20210917,
17,01101041,패션플랫폼,225590,K,[기재정정]반기보고서 (2021.06),20210917000063,패션플랫폼,20210917,
41,00957568,에스엘바이오닉스,214310,K,[기재정정]반기보고서 (2021.06),20210903000352,에스엘바이오닉스,20210903,
42,00642541,디엔에이링크,127120,K,[기재정정]사업보고서 (2020.12),20210903000235,디엔에이링크,20210903,연
44,00111218,KD,044180,K,[기재정정]반기보고서 (2021.06),20210902000297,KD,20210902,
99,01169434,오가닉티코스메틱,900300,K,반기보고서 (2021.06),20210830000797,오가닉티코스메틱,20210830,
1,01160512,헝셩그룹,900270,K,반기보고서 (2021.06),20210830000763,헝셩그룹,20210830,


In [17]:
d = dart.document('20210824000307')

In [18]:
d

'<?xml version="1.0" encoding="utf-8"?>\n\n\n<DOCUMENT xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="dart3.xsd">\n\n<DOCUMENT-HEADER AEXT-CLASS="Y">\n<DOCUMENT-NAME ACODE="11012">반기보고서</DOCUMENT-NAME>\n<FORMULA-VERSION SUBVER="1" ADATE="20210716">4.5</FORMULA-VERSION>\n<COMPANY-NAME AREGCIK="00402989">(주)코스온</COMPANY-NAME>\n\n<SUMMARY>\n<EXTRACTION ACODE="CRP_RGS_NO_TEMP" AFEATURE="BOTH">110111-1823289</EXTRACTION>\n</SUMMARY>\n</DOCUMENT-HEADER>\n\n<BODY>\n\n<INSERTION ABASISNUMBER="SE1" AFREQUENCY="0" ADUPLICATION="N">\n<COMMENT>◆click◆ 정정문서 작성시 『정오표』 삽입</COMMENT>\n<LIBRARYLIST>정정신고(보고)-대표이사등의확인.LCommon</LIBRARYLIST>\n</INSERTION>\n\n<COVER>\n<P></P>\n<COVER-TITLE ATOC="Y" AASSOCNOTE="COVER">반 기 보 고 서</COVER-TITLE>\n<P>&cr;&cr;&cr;</P>\n<P USERMARK="F-16">                                    (제 23 기)</P>\n\n<TABLE-GROUP ACLASS="COVER" ADELETETABLE="N">\n\n<TABLE ACLASS="EXTRACTION" AFIXTABLE="Y" WIDTH="600" BORDER="0">\n\n<COLGROUP>\n<COL WIDTH="